In [12]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [13]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [14]:
def search_orders_sin_factura_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    param_dia_ini = datetime(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    search_orders_sin_factura = [
        "&", "&",
            ("date_order", ">=", param_dia_ini.strftime('%Y-%m-%d')),
            ("date_order", "<=", param_dia_fin.strftime('%Y-%m-%d')),
        "&",
            ("account_move", "=", False),
            ("amount_total", "!=", 0)
        ]

    return search_orders_sin_factura

In [15]:
def api_call_pos_orders_func(api_params: dict, search_orders_sin_factura: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_orders_sin_factura = search_orders_sin_factura


    pos_doc_fields =[
                    'name',
                    'date_order',
                    'partner_id',
                    'session_id',
                    'refunded_order_ids'
                    ] 

    pos_doc_ids = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'search', [search_orders_sin_factura])
    pos_doc_json = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [pos_doc_ids], {'fields': pos_doc_fields})

    

    return pos_doc_json

In [16]:
def pos_doc_sin_factura_df_func(pos_doc_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pos_ids = []
    data_pos_refunded_ids = []

    for pos in pos_doc_json:
        new = {}
        new['pos_doc_id'] = pos['id']
        new['pos_doc_name'] = pos['name']
        new['refunded_pos_id'] = pos['refunded_order_ids'][0] if pos['refunded_order_ids'] else pd.NA
        new['pos_doc_date'] = pos['date_order']
        new['partner_id'] = pos['partner_id'][0]
        new['partner_name'] = pos['partner_id'][1]
        new['PdV_session'] = pos['session_id'][1]

        if pos['refunded_order_ids']:
            data_pos_refunded_ids.append(pos['refunded_order_ids'][0])
    
        data_pos_ids.append(new)

    pos_doc_sin_factura_df = pd.DataFrame(data_pos_ids)

    pos_doc_sin_factura_df['pos_doc_date'] = pd.to_datetime(pos_doc_sin_factura_df['pos_doc_date'], format='%Y-%m-%d %H:%M:%S')
    
    return pos_doc_sin_factura_df, data_pos_refunded_ids

In [17]:
def api_call_account_move_func(api_params: dict, data_pos_refunded_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    fact_doc_fields = [
                    'name',
                    'state',
                    'pos_order_ids'
                    ] 

    fact_doc_ids = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [[("pos_order_ids", "in", data_pos_refunded_ids)]])
    fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [fact_doc_ids], {'fields': fact_doc_fields})
  
    return fact_doc_json

In [18]:
def fact_doc_state_df_func(fact_doc_json: list[dict]) -> pd.DataFrame:
    
    data_fact_ids = []

    for fact in fact_doc_json:
        new = {}
        new['fact_doc_id'] = fact['id']
        new['fact_doc_name'] = fact['name']
        new['fact_doc_state'] = fact['state']
        new['pos_doc_id'] = fact['pos_order_ids'][0]

        data_fact_ids.append(new)

    fact_doc_state_df = pd.DataFrame(data_fact_ids)
    fact_doc_state_df['fact_doc_id'] = fact_doc_state_df['fact_doc_id'].astype('Int64')

    return fact_doc_state_df

In [19]:
api_params = api_params_func()
search_orders_sin_factura = search_orders_sin_factura_func(4)

pos_doc_json = api_call_pos_orders_func(api_params, search_orders_sin_factura)

pos_doc_sin_factura_df, data_pos_refunded_ids = pos_doc_sin_factura_df_func(pos_doc_json)

fact_doc_json = api_call_account_move_func(api_params, data_pos_refunded_ids)
fact_doc_state_df = fact_doc_state_df_func(fact_doc_json)

In [20]:
fact_doc_state_df

,fact_doc_id,fact_doc_name,fact_doc_state,pos_doc_id
0,69966,F2-CC/2024/08095,posted,14760
1,69961,F2-CC/2024/08094,posted,14759
2,69660,F2-CC/2024/08060,posted,14686
3,69007,F1-CC/2024/06156,cancel,14575
4,69001,F1-CC/2024/06154,cancel,14573
5,68803,F1-CC/2024/06132,cancel,14521
6,68278,F1-CC/2024/06095,cancel,14427
7,67796,F2-CC/2024/07885,cancel,14345
8,67245,F1-CC/2024/05990,cancel,14221
9,66008,F2-CC/2024/07676,cancel,13958


In [21]:
complete_df1 = pos_doc_sin_factura_df.merge(fact_doc_state_df, how='left', left_on='refunded_pos_id', right_on='pos_doc_id')
complete_df = complete_df1.loc[(complete_df1['fact_doc_state'] != "cancel")].drop(columns=['pos_doc_id_y'])
complete_df

,pos_doc_id_x,pos_doc_name,refunded_pos_id,pos_doc_date,partner_id,partner_name,PdV_session,fact_doc_id,fact_doc_name,fact_doc_state
0,14774,PdV SJC/8238 REEMBOLSO,14759,2024-04-26 19:09:23,16253,Mostrador,POS/00210,69961,F2-CC/2024/08094,posted
1,14772,PdV SJC/8239 REEMBOLSO,14760,2024-04-26 19:07:09,16253,Mostrador,POS/00210,69966,F2-CC/2024/08095,posted
2,14747,PdV SJC/8204 REEMBOLSO,14686,2024-04-26 18:02:58,13705,CABO BARBA CARPINTERIA,POS/00210,69660,F2-CC/2024/08060,posted
6,14560,PdV SJC/8146,<NA>,2024-04-25 17:34:48,16253,Mostrador,POS/00208,<NA>,NaN,NaN
7,14516,PdV SJC/8120,<NA>,2024-04-25 01:05:35,16253,Mostrador,POS/00206,<NA>,NaN,NaN
11,14106,PdV CSL/5910,<NA>,2024-04-22 18:27:37,16253,Mostrador,POS/00203,<NA>,NaN,NaN
13,13949,PdV SJC/7815,<NA>,2024-04-20 16:19:07,16253,Mostrador,POS/00201,<NA>,NaN,NaN
17,13687,PdV SJC/7675,<NA>,2024-04-18 19:15:04,16253,Mostrador,POS/00197,<NA>,NaN,NaN
18,13598,PdV SJC/7628,<NA>,2024-04-17 23:51:17,13428,MARCOS ROBERTO LOPEZ,POS/00195,<NA>,NaN,NaN
19,13525,PdV SJC/7589,<NA>,2024-04-17 19:10:08,16253,Mostrador,POS/00195,<NA>,NaN,NaN


In [22]:
complete_df1

,pos_doc_id_x,pos_doc_name,refunded_pos_id,pos_doc_date,partner_id,partner_name,PdV_session,fact_doc_id,fact_doc_name,fact_doc_state,pos_doc_id_y
0,14774,PdV SJC/8238 REEMBOLSO,14759,2024-04-26 19:09:23,16253,Mostrador,POS/00210,69961,F2-CC/2024/08094,posted,14759.0
1,14772,PdV SJC/8239 REEMBOLSO,14760,2024-04-26 19:07:09,16253,Mostrador,POS/00210,69966,F2-CC/2024/08095,posted,14760.0
2,14747,PdV SJC/8204 REEMBOLSO,14686,2024-04-26 18:02:58,13705,CABO BARBA CARPINTERIA,POS/00210,69660,F2-CC/2024/08060,posted,14686.0
3,14666,PdV CSL/6096 REEMBOLSO,14521,2024-04-26 00:16:02,15371,ANGEL MALDONADO TORRES,POS/00209,68803,F1-CC/2024/06132,cancel,14521.0
4,14631,PdV CSL/6118 REEMBOLSO,14573,2024-04-25 21:40:24,16253,Mostrador,POS/00209,69001,F1-CC/2024/06154,cancel,14573.0
5,14630,PdV CSL/6120 REEMBOLSO,14575,2024-04-25 21:37:43,414,Emilio Costich Perez,POS/00209,69007,F1-CC/2024/06156,cancel,14575.0
6,14560,PdV SJC/8146,<NA>,2024-04-25 17:34:48,16253,Mostrador,POS/00208,<NA>,NaN,NaN,NaN
7,14516,PdV SJC/8120,<NA>,2024-04-25 01:05:35,16253,Mostrador,POS/00206,<NA>,NaN,NaN,NaN
8,14458,PdV CSL/6059 REEMBOLSO,14427,2024-04-24 20:47:22,16253,Mostrador,POS/00207,68278,F1-CC/2024/06095,cancel,14427.0
9,14349,PdV SJC/8028 REEMBOLSO,14345,2024-04-23 23:29:12,16253,Mostrador,POS/00204,67796,F2-CC/2024/07885,cancel,14345.0
